# 3. CAPM

<p style="font-size:28px;">1. Summary Statistics</p>

In [51]:
import numpy as np
import pandas as pd
import statistics
data=pd.read_excel('dfa_analysis_data.xlsx', sheet_name='portfolios (total returns)')
START_YR=1981
df_subsample = data[(data['Date'].dt.year>= START_YR)]
df_subsample.drop(['Date'],axis=1, inplace=True)
df_subsample.dropna(inplace=True)


C:\Users\ABC\AppData\Local\Temp\ipykernel_3460\2773500845.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subsample.drop(['Date'],axis=1, inplace=True)
C:\Users\ABC\AppData\Local\Temp\ipykernel_3460\2773500845.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subsample.dropna(inplace=True)


In [52]:
data2=pd.read_excel('dfa_analysis_data.xlsx', sheet_name='factors')
data2 = data2[(data2['Date'].dt.year>= START_YR)]
df_rf=data2['RF']


In [53]:
df_excess_ret = df_subsample.sub(df_rf, axis=0)
Scale_factor=12
annualizedMean=df_excess_ret.mean()*Scale_factor        #Annualized Mean
annualizedVol=df_excess_ret.std()*np.sqrt(Scale_factor) #Annualized Volatility
sharpe=annualizedMean/annualizedVol
newData=pd.concat([annualizedMean,annualizedVol,sharpe ],axis=1)
newData.columns=['Annualized Mean', 'Annualized Volatility', 'Sharpe Ratio',]
dataSorted=df_excess_ret.sort_values(by=list(df_excess_ret.columns), na_position="first")
var=np.quantile(dataSorted, 0.05,axis=0).T
newData['Var(0.05)']=var
newData

,Annualized Mean,Annualized Volatility,Sharpe Ratio,Var(0.05)
SMALL LoBM,0.011705,0.271737,0.043075,-0.124886
ME1 BM2,0.088438,0.235431,0.375641,-0.094929
ME1 BM3,0.090226,0.200834,0.449260,-0.084817
ME1 BM4,0.112535,0.194031,0.579986,-0.077604
SMALL HiBM,0.127314,0.208354,0.611049,-0.088174
ME2 BM1,0.060922,0.244669,0.248999,-0.103232
ME2 BM2,0.098365,0.205371,0.478962,-0.083392
ME2 BM3,0.105155,0.186431,0.564042,-0.080313
ME2 BM4,0.108069,0.181868,0.594217,-0.075338
ME2 BM5,0.113192,0.213654,0.529793,-0.093275



<p style="font-size:28px;">2. CAPM</p>

In [54]:
import statsmodels.api as sm


betas = pd.Series(np.nan, index=df_excess_ret.columns)
alphas = pd.Series(np.nan, index=df_excess_ret.columns)
summed = 0
for col in df_excess_ret.columns:
    MKT =df_excess_ret[col]
    x = sm.add_constant(MKT)  
    y = df_subsample.loc[:,col]     
    model = sm.OLS(y, x).fit()
    alpha, beta = model.params["const"], model.params[col]

    betas.loc[col] = beta
    alphas.loc[col] = alpha

for i in alphas:
    summed=abs(i)+summed
treynor =annualizedMean/betas
residuals = model.resid   
ann_tracking_err = np.sqrt(Scale_factor) * residuals.std()
annualized_alphas = Scale_factor * alphas
info_ratio = annualized_alphas / ann_tracking_err

df_CAPM = pd.DataFrame({"Alpha": alpha,"Beta": betas,'Treynor':treynor,"Information Ratio":info_ratio})
print(f"Mean Absolute Error: {summed/len(df_excess_ret.columns):.6f}")
df_CAPM


Mean Absolute Error: 0.003192


,Alpha,Beta,Treynor,Information Ratio
SMALL LoBM,0.003173,0.996691,0.011744,4.037536
ME1 BM2,0.003173,0.997481,0.088661,4.057046
ME1 BM3,0.003173,0.997588,0.090445,4.056496
ME1 BM4,0.003173,0.997421,0.112827,4.064201
SMALL HiBM,0.003173,0.996445,0.127769,4.081410
ME2 BM1,0.003173,0.996319,0.061147,4.057203
ME2 BM2,0.003173,0.996933,0.098667,4.065415
ME2 BM3,0.003173,0.997307,0.105439,4.063453
ME2 BM4,0.003173,0.998330,0.108250,4.052557
ME2 BM5,0.003173,0.997882,0.113433,4.058848


In [64]:

df_crossCAPM = pd.concat([annualizedMean.rename('Excess Return'),betas.rename('Beta')], axis=1)
X = sm.add_constant(df_crossCAM['Beta'])
y = df_crossCAPM['Excess Return']

model = sm.OLS(y, X).fit()

r_squared = model.rsquared
intercept = model.params['const']
slope = model.params['Beta']

print(f"R-squared: {r_squared:.4f}")
print(f"Intercept: {intercept:.6f}")
print(f"Market Risk Premium: {slope:.6f}")

R-squared: 0.0661
Intercept: -5.345904
Market Risk Premium: 5.454447
